In [1]:
import pandas as pd
import geopandas as gpd
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer
from bokeh.plotting import figure, output_file, save
import json
from bokeh.io import curdoc, output_notebook
from bokeh.models import Slider, HoverTool
from bokeh.layouts import widgetbox, row, column
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer
from bokeh.io import curdoc, output_notebook
from bokeh.models import Slider, HoverTool
from bokeh.layouts import widgetbox, row, column

In [2]:
#Import Shape File
shapefile = "../map/ne_110m_admin_0_countries.shp"
gdf = gpd.read_file(shapefile)[['ADMIN', 'ADM0_A3', 'geometry']]
gdf.columns = ['country', 'country_code', 'geometry']
gdf.head()

,country,country_code,geometry
0,Fiji,FJI,"(POLYGON ((180 -16.06713266364245, 180 -16.555..."
1,United Republic of Tanzania,TZA,POLYGON ((33.90371119710453 -0.950000000000000...
2,Western Sahara,SAH,POLYGON ((-8.665589565454809 27.65642588959236...
3,Canada,CAN,"(POLYGON ((-122.84 49.00000000000011, -122.974..."
4,United States of America,USA,"(POLYGON ((-122.84 49.00000000000011, -120 49...."


In [3]:
#Drop Antarctica for better visualization
gdf = gdf[gdf['country'] != 'Antarctica']

In [4]:
#Import dataset
datafile = '../data/econ_freedom.csv'
data_df = pd.read_csv(datafile, names= ['year', 'country_code', 'country', 'index'], skiprows = 1)
data_df.head()


,year,country_code,country,index
0,2016,ALB,Albania,7.538571
1,2016,DZA,Algeria,4.985321
2,2016,AGO,Angola,5.166513
3,2016,ARG,Argentina,4.839536
4,2016,ARM,Armenia,7.565794


In [6]:
yr = 2016

while yr >= 2000:
    df_yr = data_df[data_df['year'] == yr]
    merged = gdf.merge(df_yr, left_on = 'country_code', right_on = 'country_code', how = 'left')
    merged.fillna('No data', inplace = True)
    merged_json = json.loads(merged.to_json())
    json_data = json.dumps(merged_json)
    geosource = GeoJSONDataSource(geojson = json_data)
    palette = brewer['PiYG'][8]
    #Reverse color order so that dark green is highest index.
    palette = palette[::-1]

    #LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
    color_mapper = LinearColorMapper(palette = palette, low = 0, high = 10, nan_color = '#d9d9d9')

    #Define custom tick labels for color bar.
    tick_labels = {'0': '0', '5': '5', '10':'10'}

    #Add hover tool
    #hover = HoverTool(tooltips = [ ('Country/region','@country_y'),('Index', '@index')])
    TOOLTIPS = [ ('Country/region','@country_y'),('Index', '@index')]
    #Create color bar. 
    color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
    border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)

    #Create figure object.
    p = figure(title = 'Economic Freedom Index, 2010', plot_height = 600 , plot_width = 950, tooltips = TOOLTIPS)
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None


    #Add patch renderer to figure. 
    p.patches('xs','ys', source = geosource,fill_color = {'field' :'index', 'transform' : color_mapper},
              line_color = 'black', line_width = 0.25, fill_alpha = 1)

    #Specify figure layout.
    p.add_layout(color_bar, 'below')
    p.title.text = 'Economic Freedom Index, %d' %yr

    #Display figure inline in Jupyter Notebook.
    output_file(p.title.text+".html")
    #Display figure.
    show(p)
    yr = yr-1
    

In [ ]:
#Filter data for year 2016.
df_yr = data_df[data_df['year'] == ]

df_2016.head()

In [ ]:
#Merge dataframes gdf and df_2016.
#Perform left merge to preserve every row in gdf.
merged = gdf.merge(df_2016, left_on = 'country_code', right_on = 'country_code', how = 'left')
#Replace NaN values to string 'No data'.
merged.fillna('No data', inplace = True)

In [ ]:
#Read data to json.
merged_json = json.loads(merged.to_json())
#Convert to String like object.
json_data = json.dumps(merged_json)

In [ ]:
#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data)

#Define a sequential multi-hue color palette.
palette = brewer['PiYG'][8]

#Reverse color order so that dark green is highest index.
palette = palette[::-1]

#LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 10, nan_color = '#d9d9d9')

#Define custom tick labels for color bar.
tick_labels = {'0': '0', '5': '5', '10':'10'}

#Add hover tool
#hover = HoverTool(tooltips = [ ('Country/region','@country_y'),('Index', '@index')])
TOOLTIPS = [ ('Country/region','@country_y'),('Index', '@index')]
#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)

#Create figure object.
p = figure(title = 'Economic Freedom Index, 2010', plot_height = 600 , plot_width = 950, tooltips = TOOLTIPS)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None


#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'index', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)

#Specify figure layout.
p.add_layout(color_bar, 'below')
p.title.text = 'Economic Freedom Index, %d' %yr

#Display figure inline in Jupyter Notebook.
output_file(p.title.text+".html")
#Display figure.
#show(p)

